In [1]:
import pandas as pd 

df = pd.read_csv('fp2.csv')

In [2]:

def clean_up(column): 
    
    new_df = df[column].values.reshape(-1,1)
    new_col = []
    
    for item in new_df:
        if type(item[0]) == str:
            
        
            col_data = item[0].split(',')
            column_lst = []
       
            for i in range(len(col_data)):
                col_data[i] = col_data[i].strip("'][ '")
                column_lst.append(col_data[i])
                
        
            new_col.append(column_lst)
    
        else: 
        
            new_col.append([])
        
        
    return new_col

In [3]:
cleanable_columns = ['Title:', 'Genre:', 'Developer:', 'Publisher:', 'Release Date:', 'Franchise:']

for item in cleanable_columns:
    clean_column = clean_up(item)
    
    df = df.drop(item, axis = 1)
    new_series = pd.Series(clean_column)
    df[item] = new_series

df = df.drop('Unnamed: 0', axis = 1)


In [4]:
def count_unique(column):
    
    unique_items = []
    
    for item in df[column]:
        for i in range(len(item)):
            
            if item[i] not in unique_items:
                
                unique_items.append(item[i])
    
    
    return unique_items
            
            
        
    

In [5]:
def one_hot(column, unique_lst):
    
    series = df[column]
    
    
    for item in unique_lst:
        
        item_check = []
        
        for i in range(len(series)):
            
            item_value = 0
            
            for j in range(len(series[i])):
                
                if item == series[i][j]:
                    item_value += 1
                else:
                    item_value += 0
                
               
            if item_value >= 1:
                item_check.append(1)
            else:
                item_check.append(0)

          
        item_series = pd.Series(item_check)
        df[item] = item_series
    



In [6]:
for item in cleanable_columns: 
    
    unique = count_unique(item)

    one_hot(item, unique)
    df = df = df.drop(item, axis = 1)
    print(item)

Title:
Genre:
Developer:
Publisher:
Release Date:
Franchise:


In [7]:
df

,Discounted Price,Number of Ratings,Percentage of Positive Reviews,Price,CounterStrikeGlobalOffensive,TheElderScrollsOnline,SidMeiersCivilizationVI,Destiny2,BlackDesertOnline,TomClancysRainbowSixSiege,...,Sins of a Solar Empire,Titan Quest,Conan Exiles,Mass Effect,Wolfenstein,Warhammer Underworlds: Online,Trine,Surviving Mars,Power Rangers,MotoGP™
0,NaN,4427442.0,87%,Free to Play,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,60316.0,81%,$19.99,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,76894.0,80%,$59.99,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,237260.0,86%,Free To Play,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,24355.0,74%,$9.99,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,NaN,50.0,70%,$9.99,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
941,NaN,69.0,66%,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
942,NaN,38384.0,91%,$19.99,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
943,NaN,144.0,79%,$14.99,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df.to_csv("new_data")